In [1]:
import pandas as pd
import re
import numpy as np

In [2]:
df_movies_features = pd.read_pickle("movie_features_v2.pkl")
df_movies_features.head()

,movieId,description_encoded,genre_encoded,year_encoded,content_rating_encoded,year_binned,length_binned
movieId,,,,,,,
1,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0, 0, 0]","[0, 1, 0, 0, 0, 0]",1995,80
2,2,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0, 0, 0]","[0, 0, 0, 1, 0, 0]",1995,80
3,3,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0, 0, 0]","[1, 0, 0, 0, 0, 0]",1995,80
4,4,"[0.0, 0.0, 0.0, 0.0, 0.37513907199, 0.0, 0.0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...","[0, 1, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 1]",1995,120
5,5,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0, 0, 0]","[0, 0, 0, 1, 0, 0]",1995,80


In [3]:
movie_ids = set(df_movies_features["movieId"].unique())

In [4]:
df_ratings = pd.read_csv("ml-20m/ratings.csv")
df_ratings.describe()

,userId,movieId,rating,timestamp
count,2.000026e+07,2.000026e+07,2.000026e+07,2.000026e+07
mean,6.904587e+04,9.041567e+03,3.525529e+00,1.100918e+09
std,4.003863e+04,1.978948e+04,1.051989e+00,1.621694e+08
min,1.000000e+00,1.000000e+00,5.000000e-01,7.896520e+08
25%,3.439500e+04,9.020000e+02,3.000000e+00,9.667977e+08
50%,6.914100e+04,2.167000e+03,3.500000e+00,1.103556e+09
75%,1.036370e+05,4.770000e+03,4.000000e+00,1.225642e+09
max,1.384930e+05,1.312620e+05,5.000000e+00,1.427784e+09


In [5]:
# filter by movieId
df_ratings = df_ratings[df_ratings["movieId"].apply(lambda x: x in movie_ids)]

In [6]:
# fitler by timestamp
#2005 to #2015
df_ratings2 = df_ratings[df_ratings["timestamp"]>=1325376000]

In [7]:
len(df_ratings2.userId.unique())

19576

In [8]:
L = (df_ratings2.groupby("userId").count()["movieId"]>=20)
users_set = L[L.values].index
len(users_set)

14725

In [9]:
import random
users_set = set(random.sample(set(users_set),7000))

In [10]:
df_ratings2 = df_ratings2[df_ratings2["userId"].apply(lambda x: x in users_set)]

In [11]:
df_ratings2.shape

(724217, 4)

In [12]:
len(df_ratings2.movieId.unique()),len(df_ratings2.userId.unique())

(8492, 7000)

df_ratings2.columns=["UserID","ItemID","Rating","Timestamp"]

df_ratings2.head()

df_ratings2.to_csv("movielens20M.ratings.data.csv", index=None)
df_ratings2.to_pickle("data/movielens20M.ratings.data.pkl")

df_ratings2.groupby("ItemID").count().describe()

df_ratings2.UserID.max()

### Id remapping

df_movies_features = pd.read_pickle("movie_features_v.pkl")

In [16]:
df_movies_features.head()

,movieId,description_encoded,genre_encoded,year_encoded,content_rating_encoded,year_binned,length_binned
movieId,,,,,,,
1,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0, 0, 0]","[0, 1, 0, 0, 0, 0]",1995,80
2,2,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0, 0, 0]","[0, 0, 0, 1, 0, 0]",1995,80
3,3,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0, 0, 0]","[1, 0, 0, 0, 0, 0]",1995,80
4,4,"[0.0, 0.0, 0.0, 0.0, 0.37513907199, 0.0, 0.0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...","[0, 1, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 1]",1995,120
5,5,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0, 0, 0]","[0, 0, 0, 1, 0, 0]",1995,80


In [13]:
df_ratings2.head()

,userId,movieId,rating,timestamp
7025,60,31,4.0,1339651304
7026,60,157,4.5,1339653445
7027,60,256,2.0,1339651256
7028,60,277,2.5,1339651275
7029,60,296,3.5,1339652690


In [23]:
item_set = set(df_ratings2.movieId.unique())
len(item_set)

8492

In [24]:
i = 0
id_map_movie = dict()
for m in item_set:
    if not m in id_map_movie:
        id_map_movie[m]=i
        i+=1

In [25]:
df_movies_features = df_movies_features[df_movies_features.movieId.apply(lambda x: x in item_set)]

In [26]:
df_movies_features["movieId_mapped"] = df_movies_features["movieId"].apply(lambda x: id_map_movie[x])

In [27]:
df_movies_features.index = df_movies_features["movieId_mapped"]

In [33]:
df_movies_features.head()

,movieId,description_encoded,genre_encoded,year_encoded,content_rating_encoded,year_binned,length_binned,movieId_mapped
movieId_mapped,,,,,,,,
1,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0, 0, 0]","[0, 1, 0, 0, 0, 0]",1995,80,1
2,2,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0, 0, 0]","[0, 0, 0, 1, 0, 0]",1995,80,2
3,3,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0, 0, 0]","[1, 0, 0, 0, 0, 0]",1995,80,3
4,4,"[0.0, 0.0, 0.0, 0.0, 0.37513907199, 0.0, 0.0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...","[0, 1, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 1]",1995,120,4
5,5,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0, 0, 0]","[0, 0, 0, 1, 0, 0]",1995,80,5


In [34]:
df_ratings2["movieId_mapped"] = df_ratings2["movieId"].apply(lambda x: id_map_movie[x])

In [35]:
i = 0
id_map_user = dict()
for m in df_ratings2["userId"].unique():
    if not m in id_map_user:
        id_map_user[m]=i
        i+=1

In [36]:
df_ratings2["userId_mapped"] = df_ratings2["userId"].apply(lambda x: id_map_user[x])

In [37]:
df_ratings2.head()

,userId,movieId,rating,timestamp,movieId_mapped,userId_mapped
7025,60,31,4.0,1339651304,31,0
7026,60,157,4.5,1339653445,139,0
7027,60,256,2.0,1339651256,235,0
7028,60,277,2.5,1339651275,254,0
7029,60,296,3.5,1339652690,272,0


In [38]:
df_ratings2.columns = ["userId_org","movieId_org","Rating","Timestamp","ItemID","UserID"]
df_ratings2.head()

,userId_org,movieId_org,Rating,Timestamp,ItemID,UserID
7025,60,31,4.0,1339651304,31,0
7026,60,157,4.5,1339653445,139,0
7027,60,256,2.0,1339651256,235,0
7028,60,277,2.5,1339651275,254,0
7029,60,296,3.5,1339652690,272,0


In [39]:
df_ratings2.to_pickle("data/movielens20M.ratings.data.pkl")

In [40]:
df_movies_features.head()

,movieId,description_encoded,genre_encoded,year_encoded,content_rating_encoded,year_binned,length_binned,movieId_mapped
movieId_mapped,,,,,,,,
1,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0, 0, 0]","[0, 1, 0, 0, 0, 0]",1995,80,1
2,2,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0, 0, 0]","[0, 0, 0, 1, 0, 0]",1995,80,2
3,3,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0, 0, 0]","[1, 0, 0, 0, 0, 0]",1995,80,3
4,4,"[0.0, 0.0, 0.0, 0.0, 0.37513907199, 0.0, 0.0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...","[0, 1, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 1]",1995,120,4
5,5,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0, 0, 0]","[0, 0, 0, 1, 0, 0]",1995,80,5


In [41]:
df_movies_features.columns = ['movieId_org', 'Description', 'Genre', 'Year',
       'ContentRating', 'Year_B', 'Length_B','ItemID']
df_movies_features.head()

,movieId_org,Description,Genre,Year,ContentRating,Year_B,Length_B,ItemID
movieId_mapped,,,,,,,,
1,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0, 0, 0]","[0, 1, 0, 0, 0, 0]",1995,80,1
2,2,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0, 0, 0]","[0, 0, 0, 1, 0, 0]",1995,80,2
3,3,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0, 0, 0]","[1, 0, 0, 0, 0, 0]",1995,80,3
4,4,"[0.0, 0.0, 0.0, 0.0, 0.37513907199, 0.0, 0.0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...","[0, 1, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 1]",1995,120,4
5,5,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0, 0, 0]","[0, 0, 0, 1, 0, 0]",1995,80,5


### Counts of items

In [42]:
item_counts = df_ratings2.groupby("ItemID").count()["UserID"]

In [43]:
df_ratings2.ItemID.max()

8491

In [44]:
df_movies_features["count"] = df_movies_features.ItemID.apply(lambda x: item_counts.loc[x])

In [47]:
df_movies_features.head()

,movieId_org,Description,Genre,Year,ContentRating,Year_B,Length_B,ItemID,count
movieId_mapped,,,,,,,,,
1,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0, 0, 0]","[0, 1, 0, 0, 0, 0]",1995,80,1,1809
2,2,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0, 0, 0]","[0, 0, 0, 1, 0, 0]",1995,80,2,619
3,3,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0, 0, 0]","[1, 0, 0, 0, 0, 0]",1995,80,3,100
4,4,"[0.0, 0.0, 0.0, 0.0, 0.37513907199, 0.0, 0.0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...","[0, 1, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 1]",1995,120,4,16
5,5,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0, 0, 0]","[0, 0, 0, 1, 0, 0]",1995,80,5,235


In [48]:
df_movies_features.to_pickle("data/movielens20M.itemfeatures.pkl")

In [ ]:
df_movies_features.head()

### Test-train split

df = df_ratings2
df.columns=["UserID","ItemID","Rating","Timestamp"]

df = df.sort_values("Timestamp", ascending=False)
df_ratings.head()

from collections import defaultdict

count_per_user = 1
count_per_user_valid = 0
user_counts = defaultdict(int)
rows = []
rows_valid = []
for index, row in df.iterrows():
    if user_counts[row["UserID"]] < count_per_user:
        rows += [index]
        user_counts[row["UserID"]] +=1
    elif user_counts[row["UserID"]] < count_per_user+count_per_user_valid:
        rows_valid += [index]
        #user_counts[row["UserID"]] +=1

df_test_ratings = df.loc[rows]
df_train_ratings = df.loc[list(set(df.index)-set(df_test_ratings.index))]

df_test_ratings = df_test_ratings.sort_values(["UserID","ItemID"])
df_test_ratings.head()

df_train_ratings = df_train_ratings.sort_values(["UserID","ItemID"])
df_train_ratings.head()

df_test_ratings.to_pickle("movielens20M.test.data")
df_train_ratings.to_pickle("movielens20M.train.data")

!ls movielens20M*

### Negatives

%%time
user_item_pairs = defaultdict(set)

for x in df[["UserID", "ItemID"]].values:
    # add all +ve to set
    user_item_pairs[x[0]].add(x[1])

num_items = df["ItemID"].max()

def get_negs(u):
    negs = []
    for _i in range(num_negs_per_positive):
        # generate negatives and add to dict
        j = np.random.randint(num_items)
        while (u,j) in user_item_pairs[u]:
            j = np.random.randint(num_items)
        negs += [j]
        #user_item_pairs[u].add(j)
    return negs

num_negs_per_positive = 99
df_test_ratings["Negatives"] = df_test_ratings["UserID"].apply(lambda x: get_negs(x))
#num_negs_per_positive = 99
#df_valid_ratings["Negatives"] = df_valid_ratings["UserID"].apply(lambda x: get_negs(x))
for x in df_test_ratings[["UserID", "ItemID"]].values:
    # add all +ve to set
    user_item_pairs[x[0]].add(x[1])

num_negs_per_positive = 7
df_train_ratings["Negatives"] = df_train_ratings["UserID"].apply(lambda x: get_negs(x))

len(df_ratings2.UserID.unique()),len(df_ratings2.ItemID.unique()),

df_ratings2.describe()

df_merge = df_ratings2.merge(df_movies_features, left_on="ItemID", right_on="movieId")


df_merge = df_merge.drop("movieId",axis=1)

df_merge.columns = ['UserID', 'ItemID', 'Rating', 'Timestamp', 'Description',
       'Genre', 'Year']

df_merge.index = df_merge["ItemID"]

df_merge.to_pickle("data/movielens20M.ratings.data.pkl")

df_ratings2.UserID.max()

df_movies_features["count"].value_counts()

### Cold start sampling

items = df_movies_features.ItemID.unique()

test_set_items = set(random.sample(set(items), int(len(items)/10)))

df_test_cold_start = df_ratings2[df_ratings2.ItemID.apply(lambda x: x in test_set_items)]

df_test_cold_start.shape

df_train = df_ratings2.loc[list(set(df_ratings2.index) - set(df_test_cold_start.index))]

set(df_test_cold_start.UserID.unique())-set(df_train.UserID.unique())

type(df_train.loc[[18340311,18340312,18340313]]["UserID"].values)

np.array([df_movies_features.loc[[0,1,2]]["Description"].values[0],df_movies_features.loc[[0,1,2]]["Description"].values[1]]).shape


np.array(df_movies_features.loc[[0,1,2]]["Description"].tolist()).shape